In [3]:
import pandas as pd
import numpy as np
import requests
import time
import re

from tqdm import tqdm
from bs4 import BeautifulSoup

## Функции

In [4]:
columns_to_handle = ['bodyType', 'brand', 'car_url', 'color', 'engineDisplacement', 'enginePower', 
                      'equipment_dict','fuelType', 'mileage', 'modelDate', 'model_name', 'numberOfDoors', 
                      'parsing_unixtime', 'productionDate', 'sell_id', 'vehicleTransmission', 'vendor', 
                      'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль']

In [5]:
def get_marks_models():
    '''
    возврашщает pd.Series в котором 
    индекс - название марки автомобиоя, 
    значения - списки названий моделей для каждой маркию
    
    '''
    marks_models = pd.Series() 

    for pages_num in range(1,20):
        if pages_num==1: params = {'view_type': 'list'}
        else:            params = {'page_num': pages_num,'view_type': 'list'}
    
        res = requests.get(url_for_marks_models, params=params, headers=headers_for_marks_models)
        time.sleep(0.5)
    
        soup = BeautifulSoup(res.text, 'html.parser')  # создаем обьект bs4.BeautifulSoup
        marks_on_page_list = soup.find_all('dd', class_='catalog-all-text-list__desc') # список html-ек марок на странице
    
        if not marks_on_page_list: 
            break
        
        for mark in marks_on_page_list:
            models_of_mark = mark.find_all('a', class_='link_theme_auto') # список html-ек моделей марки
        
            link_for_mark_name = models_of_mark[0].get('href')  # линк первой модели, содержит обозначене марки на auto.ru
            mark_start = link_for_mark_name.find('cars/') + 5   # первый символ обозначеня марки
            mark_end = link_for_mark_name.find('/', mark_start) # последний символ обозначения марки
            mark_name = link_for_mark_name[mark_start:mark_end].upper() # получение обозначения марки      
        
            models_list = []
            for model in models_of_mark:
                link_theme_auto = model.get('href')                 # линк модели, содержит обозначение модели на auto.ru
                model_start = mark_end+1                            # первый символ обозначения модели
                model_end = link_theme_auto.find('/', model_start)  # последний символ обозначения модели
                model_name = link_theme_auto[model_start:model_end].upper() # получение обозначения модели
                models_list.append(model_name)
      
            marks_models[mark_name] = models_list
    return marks_models

def get_generation_year(model_url):
    '''
    возврашщает pd.Series  в котором 
        индекс - полное название модели с указанием поколения, 
        значение - год начала выпуска поколения
        
    model_url - ссылка на страницу со списокм поколений модели
    
    '''    
    # получем html 
    models_list_res = requests.get(model_url+'?output_type=models_list')
    models_list_res.encoding = 'utf-8'
    
    # создаем обьект bs4.BeautifulSoup из html 
    models_list_bs = BeautifulSoup(models_list_res.text, 'html.parser') 
    
    # получаем список полных названий модели с указание поколения и список годов начала выпуска поколения
    full_name_tag = models_list_bs.find_all('a',class_ = 'ListingItemTitle-module__link')
    generation_years_tag = models_list_bs.find_all('div',class_ = "ListingItemTitle-module__subtitle")
    
    data=[int(x.text[x.text.find('(')+1:x.text.find('(')+5])  for x in generation_years_tag]
    index=[x.text for x in full_name_tag][:len(data)] # иногда бывает модель без поколения и года - обрезаем

    return pd.Series(index=index,data=data)

def get_model_generation_year(marks_models_for_parsing):
    '''
    возвращает pd.Series в котором
        индекс: марки и модели данной марки автомобиля с указанием поколения модели
        значение: год начала выпуска поколоения модели
    
    marks_models_for_parsing - pd.Series в котором
        индекс: марка автомобиля
        значение: список моделей данной маркеи
    '''
    model_generation_year = pd.Series()
    
    for mark in marks_models_for_parsing.index:
        print(mark)
        for model in marks_models_for_parsing[mark]:
            model_url = 'https://auto.ru/moskva/cars/' + mark.lower() + '/' + model.lower() + '/used/'
            tmp = get_generation_year(model_url)
            model_generation_year = model_generation_year.append(tmp)
            time.sleep(0.5)
    return model_generation_year


columns_to_handle = ['bodyType', 'brand', 'car_url', 'color', 'engineDisplacement', 'enginePower', 
                      'equipment_dict','fuelType', 'mileage', 'modelDate', 'model_name', 'numberOfDoors', 
                      'productionDate', 'sell_id', 'vehicleTransmission', 'vendor', 
                      'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль']

def get_features_from_ticket(ticket_url):
    '''
    возвращает pd.Series с признакамии полученными из карточки обьявления
    
    ticket_url: str, ссылка на страницу обьявления
        
    '''
    features = pd.Series(index = data_columns)
    
    # получем html карточки текщего обьявления
    ticket_res = requests.get(ticket_url)
    ticket_res.encoding = 'utf-8'
    
    # создаем обьект bs4.BeautifulSoup из html карточки текщего обьявления
    ticket_bs = BeautifulSoup(ticket_res.text, 'html.parser')  
    # проверка корректности резкльтата BeautifulSoup
    if ticket_bs:
    
    # получение признаков из карточки текущего обьявления
    #
        # bodytype      
        try: features['bodytype'] = ticket_bs.find('li',class_='CardInfoRow_bodytype').find('a').text
        except Exception: features['bodytype'] = np.NaN
           
        # brand        
        try: features['brand'] = ticket_bs.find_all('a',class_='CardBreadcrumbs__itemText')[0].text.strip().upper()
        except Exception: features['brand'] = np.NaN
    
        # car_url
        features['car_url'] = ticket_url
    
        # color        
        try: features['color'] = ticket_bs.find('li',class_='CardInfoRow_color').find('a').text
        except Exception: features['color'] = np.NaN
    
        #engineDisplacement            
        try: 
            engineDisplacement = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[0]
            features['engineDisplacement'] =  re.sub("[^\d.]", "", engineDisplacement)
        except Exception: features['engineDisplacement'] = np.NaN
          
        # enginePower
        try: 
            enginePower = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[1]
            features['enginePower'] = re.sub("\D", "", enginePower)
        except Exception: features['enginePower'] = np.NaN
    
        # equipment_dict
        equipment_dict = dict()
        try:
            complectation = ticket_bs.find('section',class_='CardComplectation')
            complectation_items = complectation.find_all('div',class_="ComplectationGroups__group")
            for item in complectation_items:
                item_name = item.find('span',class_="ComplectationGroups__itemName").text
                item_content_tag = item.find_all('li',class_="ComplectationGroups__itemContentEl")
                item_content = [x.text for x in item_content_tag]
                equipment_dict[item_name] = item_content 
            features['equipment_dict'] = equipment_dict
        except Exception: features['equipment_dict'] = np.NaN
    
        # fuel_type
        try: features['fuel_type'] = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[2]
        except Exception: features['fuel_type'] = np.NaN
        
        # mileage
        try:
            mileage = ticket_bs.find('li',class_='CardInfoRow_kmAge').find_all('span')[1].text
            features['mileage'] = re.sub("\D", "", mileage)
        except Exception: features['mileage'] = np.NaN
    
        # modelDate
        ## заполняется названием марки, модели и поколением модели - следует далее заменить на год модели/поколения
        try:
            modelDate_tag = ticket_bs.find_all('a',class_='CardBreadcrumbs__itemText')
            features['modelDate'] = (modelDate_tag[0].text.strip() + ' ' +
                                     modelDate_tag[1].text.strip() + ' ' +
                                     modelDate_tag[2].text.strip())  
        except Exception: features['modelDate'] = np.NaN
    
        # model_name
        try: features['model_name'] = ticket_bs.find_all('a',class_='CardBreadcrumbs__itemText')[1].text.strip()
        except Exception: features['model_name'] = np.NaN
     
        # numberOfDoors
        try:
            numberOfDoors_tag = ticket_bs.find('li',class_='CardInfoRow_bodytype').find('a')
            numberOfDoors_pre = re.findall('\d', numberOfDoors_tag.text)
            features['numberOfDoors'] = int(numberOfDoors_pre[0])
        except Exception: features['numberOfDoors'] = np.NaN
    
        # productionDate
        try: features['productionDate'] = ticket_bs.find('li',class_='CardInfoRow_year').find('a').text
        except Exception: features['productionDate'] = np.NaN
    
        # sell_id
        try:         
            invers_ticket_url = ticket_url[::-1]
            id_start, id_end = invers_ticket_url.find('/',1) , invers_ticket_url.find('-')+1
            features['sell_id'] = ticket_url[-id_start:-id_end]
        except Exception: features['sell_id'] = np.NaN
    
        # vehicleTransmission
        try:
            features['vehicleTransmission'] = (ticket_bs.find('li',class_='CardInfoRow_transmission').
                                               find_all('span')[1].text)
        except Exception: features['vehicleTransmission'] = np.NaN
            
        # vendor
        european = ['SKODA', 'AUDI',  'VOLVO', 'BMW', 'MERCEDES', 'VOLKSWAGEN']
        japanese = ['HONDA','NISSAN','TOYOTA','INFINITI',  'LEXUS', 'MITSUBISHI']
        if features['brand'] in european :  features['vendor'] = 'EUROPEAN'
        elif features['brand'] in japanese :  features['vendor'] = 'JAPANESE'
        else: features['vendor'] = 'NAN'
        
        # Владельцы
        try: features['Владельцы'] = ticket_bs.find('li',class_='CardInfoRow_ownersCount').find_all('span')[1].text
        except Exception: features['Владельцы'] = np.NaN

        # ПТС
        try: features['ПТС'] = ticket_bs.find('li',class_='CardInfoRow_pts').find_all('span')[1].text
        except Exception: features['ПТС'] = np.NaN
        
        # Привод
        try: features['Привод'] = ticket_bs.find('li',class_='CardInfoRow_drive').find_all('span')[1].text
        except Exception: features['Привод'] = np.NaN
    
        # Руль
        try: features['Руль'] = ticket_bs.find('li',class_='CardInfoRow_wheel').find_all('span')[1].text 
        except Exception: features['Руль'] = np.NaN
    
        # Привод
        try: features['Привод'] = ticket_bs.find('li',class_='CardInfoRow_drive').find_all('span')[1].text
        except Exception: features['Привод'] = np.NaN

        # Цена предложения
        try:
            offerprice = ticket_bs.find('span',class_='OfferPriceCaption__price').text
            features['offerprice'] = re.sub("\D", "", offerprice)
        except Exception: features['offerprice'] = np.NaN
            
    return features

## РАЗРАБОТКА =====================================

## РАЗРАБОТКА =====================================

### Создаем и наполняем ***`marks_models `***  
***marks_models*** - pd.Series в которм   
индекс:  обозначения марки на сайте auto.ru  
значения: списки обозначений на сайте auto.ru марок у каждой модели.

In [6]:
# определяем константы

url_for_marks_models = 'https://auto.ru/catalog/cars/all/'

headers_for_marks_models = {
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
'Cache-Control': 'max-age=0',
'Connection': 'keep-alive',
'Cookie': 'suid=bf4a59ff8840503c2077bf716a7bbeef.295e234731ada0bc538a541689345348; tmr_lvid=d2115cf4bc0ee3e6bc3ad89db8fdac9d; tmr_lvidTS=1596630011564; _ym_uid=1596630015684249973; _ga=GA1.2.1113974574.1596630016; autoruuid=g5f949c622cu74bpb1qps6a12l6p5u7t.07262c19ce2a1f07cac53c4ce06ef69b; gids=213; gradius=200; mindboxDeviceUUID=a231f610-2f9e-4911-9cb8-9112afa13ab1; directCrm-session=%7B%22deviceGuid%22%3A%22a231f610-2f9e-4911-9cb8-9112afa13ab1%22%7D; tmr_reqNum=15; yuidlt=1; yandexuid=1471619051363959249; my=YysBgNU2AQEA; crookie=uCcId3jGlYfIvH+2UzzszOwTawIMGSzWumxanVb1Ras+Mu6qi+8yzj8EL+czftU9orxvkiKDKe/wgGsJDdJMlqyn6WU=; cmtchd=MTYxMjgyMjU3MTQ4MA==; _csrf_token=04c10009d8f17a08b4d507f70f2a287c28bf3c17f7b3ef42; gdpr=0; _ym_isad=2; index-selector-tab=marks; listing_view_session={}; listing_view=%7B%22output_type%22%3Anull%2C%22version%22%3A1%7D; autoru-visits-count=2; salon_phone_utms=utm_medium%3Dcpm%26utm_source%3Dauto-ru%26utm_campaign%3Dauto-ru_rus-r225_proauto-rk2021%26utm_content%3D113pa-100PRx40-otchety-o-proshlom-mashiny-ot-99-rublei_proauto-promo-page_rus-r225; hide-proauto-pimple=1; from=direct; autoru_sid=a%3Ag5f949c622cu74bpb1qps6a12l6p5u7t.07262c19ce2a1f07cac53c4ce06ef69b%7C1613427368824.604800.BYczsuAhOkO7E_tI9WN3ZQ.vyyvypdKR_sNAGlonPspJxUInUX2GwuD7owqbI5Sw58; X-Vertis-DC=vla; _ym_d=1613250934; from_lifetime=1613250934966; cycada=FXXHRKJxPTj6XyBIJ1I0Or150N9cGJGcL1yh7v8BaXc=',
'Host': 'auto.ru',
'sec-ch-ua': '"Chromium";v="88", "Google Chrome";v="88", ";Not A Brand";v="99"',
'sec-ch-ua-mobil': '?0',
'Sec-Fetch-Dest': 'document',
'Sec-Fetch-Mode': 'navigate',
'Sec-Fetch-Site': 'same-origin',
'Sec-Fetch-User': '?1',
'Upgrade-Insecure-Requests': '1',
'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
};

In [7]:
marks_models = get_marks_models()

In [8]:
marks_for_parsing = ['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI',
       'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI']
marks_models_for_parsing = marks_models.loc[marks_for_parsing]

In [13]:
extra_marks = marks_models.index.drop(marks_for_parsing)
extra_marks_models = marks_models.loc[extra_marks]

#### Создаем и заполняем `model_generation_year` - pd.Series в котором 
индекс: марки и модели данной марки автомобиля с указанием поколения модели  
значение: год начала выпуска поколоения модели

In [258]:
%%time
# model_generation_year =  get_model_generation_year(marks_models_for_parsing)
# model_generation_year.to_csv('./Project_5_data/model_generation_year')
model_generation_year = pd.read_csv('./Project_5_data/model_generation_year')

Wall time: 216 ms


# Парсинг
### Инициализируем `df_train`

In [23]:
data_columns = ['bodytype', 'brand', 'car_url', 'color', 'engineDisplacement',
       'enginePower', 'equipment_dict', 'fuel_type', 'mileage', 'modelDate',
       'model_name', 'numberOfDoors', 'productionDate', 'sell_id', 'vehicleTransmission',
       'vendor', 'Владельцы', 'ПТС', 'Привод', 'Руль', 'offerprice']

In [265]:
train = pd.DataFrame(columns=data_columns)

### Цикл по всем моделям всех марок

In [267]:
%%time
# for mark in tqdm(marks_models_for_parsing.index):
for mark in marks_models_for_parsing.index:
    for model in marks_models_for_parsing[mark]:
        model_url = 'https://auto.ru/moskva/cars/' + mark + '/' + model + '/used/'
        print(mark, model, len(train))

    # просмотр последовательно всех страниц текущей модели-model текущей марки-mark
        for pages_num in range(1,100): 
        
            if pages_num==1: params = {}
            else:            params = {'page': pages_num}
    
            res = requests.get(model_url + '?output_type=table', params=params, headers=headers_for_marks_models)
            res.encoding = 'utf-8'
            time.sleep(0.5)
    
            # создаем обьект bs4.BeautifulSoup из html очередной страницы с обьявлениями по текущей модели текущей марки
            soup = BeautifulSoup(res.text, 'html.parser')  
    
            # список html-ек карточек на странице                                               
            tickets_on_page_list = soup.find_all('a', class_='ListingItemTitle-module__link') 
    
            if not tickets_on_page_list: # выход по исчерпанию страниц текущей модели текущей марки
                break
        
            # обработка карточек на странице
            for ticket in tickets_on_page_list:
                # получаем url карточки текущего обьявления
                ticket_url = ticket.get('href')
    
                # извлекаем признаки и заполняем строку train-а
                train.loc[len(train)] = get_features_from_ticket(ticket_url)



SKODA 120 0
SKODA 1200 1
SKODA FABIA 2
SKODA FABIA_RS 153
SKODA FAVORIT 154
SKODA FELICIA 156
SKODA KAROQ 184
SKODA KODIAQ 201
SKODA OCTAVIA 291
SKODA OCTAVIA_RS 1311
SKODA RAPID 1332
SKODA ROOMSTER 1668
SKODA SUPERB 1686
SKODA YETI 1874
AUDI 100 2055
AUDI 200 2135
AUDI 80 2139
AUDI 90 2209
AUDI 920 2216
AUDI A1 2217
AUDI A3 2239
AUDI A4 2413
AUDI A4_ALLROAD 2796
AUDI A5 2809
AUDI A6 3007
AUDI A7 3576
AUDI A8 3657
AUDI ALLROAD 3816
AUDI E_TRON 3864
AUDI Q3 3868
AUDI Q3_SPORTBACK 3990
AUDI Q5 3992
AUDI Q7 4155
AUDI Q8 4405
AUDI R8 4434
AUDI RS3 4439
AUDI RS4 4441
AUDI RS5 4445
AUDI RS6 4450
AUDI RS7 4458
AUDI RS_Q8 4459
AUDI RSQ3 4459
AUDI S3 4469
AUDI S4 4471
AUDI S5 4476
AUDI S6 4489
AUDI S7 4491
AUDI S8 4493
AUDI SQ5 4495
AUDI SQ7 4497
AUDI SQ8 4497
AUDI TT 4497
HONDA ACCORD 4525
HONDA ACTY 4706
HONDA ASCOT 4709
HONDA AVANCIER 4711
HONDA BEAT 4716
HONDA CIVIC 4717
HONDA CIVIC_FERIO 4919
HONDA CIVIC_TYPE_R 4920
HONDA CONCERTO 4924
HONDA CR_V 4925
HONDA CR_X 5083
HONDA CR_Z 5084
HONDA 

TOYOTA TUNDRA 22426
TOYOTA VANGUARD 22461
TOYOTA VELLFIRE 22462
TOYOTA VENZA 22463
TOYOTA VEROSSA 22483
TOYOTA VERSO 22485
TOYOTA VISTA 22498
TOYOTA VITZ 22505
TOYOTA VOLTZ 22516
TOYOTA VOXY 22518
TOYOTA WILL 22526
TOYOTA WINDOM 22527
TOYOTA WISH 22532
TOYOTA YARIS 22534
LEXUS CT 22562
LEXUS ES 22566
LEXUS GS 22671
LEXUS GX 22749
LEXUS IS 22800
LEXUS LC 22852
LEXUS LM 22853
LEXUS LS 22854
LEXUS LX 22911
LEXUS NX 23082
LEXUS RC 23163
LEXUS RX 23165
LEXUS SC 23405
LEXUS UX 23410
VOLKSWAGEN AMAROK 23412
VOLKSWAGEN ARTEON 23457
VOLKSWAGEN BEETLE 23458
VOLKSWAGEN BORA 23480
VOLKSWAGEN CADDY 23511
VOLKSWAGEN CARAVELLE 23605
VOLKSWAGEN CORRADO 23720
VOLKSWAGEN EOS 23723
VOLKSWAGEN EUROVAN 23724
VOLKSWAGEN FOX 23724
VOLKSWAGEN GOLF 23725
VOLKSWAGEN GOLF_COUNTRY 24040
VOLKSWAGEN GOLF_GTI 24041
VOLKSWAGEN GOLF_PLUS 24059
VOLKSWAGEN GOLF_R 24110
VOLKSWAGEN GOLF_R32 24115
VOLKSWAGEN ILTIS 24116
VOLKSWAGEN JETTA 24117
VOLKSWAGEN LUPO 24408
VOLKSWAGEN MULTIVAN 24411
VOLKSWAGEN PASSAT 24547
VOLKSWAGE

In [268]:
# train.to_csv('./Project_5_data/train')

In [269]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29075 entries, 0 to 29074
Data columns (total 21 columns):
bodytype               29067 non-null object
brand                  29069 non-null object
car_url                29075 non-null object
color                  29069 non-null object
engineDisplacement     28991 non-null object
enginePower            28991 non-null object
equipment_dict         22854 non-null object
fuel_type              28991 non-null object
mileage                29069 non-null object
modelDate              29067 non-null object
model_name             29069 non-null object
numberOfDoors          13863 non-null object
productionDate         29069 non-null object
sell_id                29075 non-null object
vehicleTransmission    29067 non-null object
vendor                 29075 non-null object
Владельцы              29067 non-null object
ПТС                    29069 non-null object
Привод                 29067 non-null object
Руль                   29069 non-nul

## Парсинг extra_marks

In [27]:
extra_train = pd.DataFrame(columns=data_columns)

In [28]:
%%time
# for mark in tqdm(marks_models_for_parsing.index):
for mark in extra_marks_models.loc['KIA':].index:
    print(mark, len(extra_train), end='\n\t')
    for model in extra_marks_models.loc['KIA':][mark]:
        model_url = 'https://auto.ru/moskva/cars/' + mark + '/' + model + '/used/'
        print(model, end = ' ')

    # просмотр последовательно всех страниц текущей модели-model текущей марки-mark
        for pages_num in range(1,100): 
        
            if pages_num==1: params = {}
            else:            params = {'page': pages_num}
    
            res = requests.get(model_url + '?output_type=table', params=params, headers=headers_for_marks_models)
            res.encoding = 'utf-8'
            time.sleep(0.5)
    
            # создаем обьект bs4.BeautifulSoup из html очередной страницы с обьявлениями по текущей модели текущей марки
            soup = BeautifulSoup(res.text, 'html.parser')  
    
            # список html-ек карточек на странице                                               
            tickets_on_page_list = soup.find_all('a', class_='ListingItemTitle-module__link') 
    
            if not tickets_on_page_list: # выход по исчерпанию страниц текущей модели текущей марки
                break
        
            # обработка карточек на странице
            for ticket in tickets_on_page_list:
                # получаем url карточки текущего обьявления
                ticket_url = ticket.get('href')
    
                # извлекаем признаки и заполняем строку train-а
                extra_train.loc[len(extra_train)] = get_features_from_ticket(ticket_url)
    print()



KIA 0
	AVELLA CARENS CARNIVAL CEED CEED_GT CERATO CLARUS FORTE JOICE K5 K900 MAGENTIS MOHAVES OPIRUS OPTIMA PICANTO PRIDE PROCEED QUORIS RAY RETONA RIO SEDONA SELTOS SEPHIA SHUMA SORENTO SOUL SOUL_EV SPECTRA SPORTAGE STINGER VENGA XCEED XCEED VAZ 4302
	1111 2101 2102 2103 2104 2105 2106 2107 2108 2109 21099 2110 2111 2112 2113 2114 2115 2120 2121 2129 2131_4X4 2170 2329 ELLADA GRANTA KALINA LARGUS NIVA NIVA_LEGEND VESTA XRAY XRAY LAMBORGHINI 9701
	AVENTADOR GALLARDO HURACAN LM002 MURCIELAGO MURCIELAGO LANCIA 9709
	DELTA KAPPA THESIS THESIS LAND_ROVER 9713
	DEFENDER DISCOVERY DISCOVERY_SPORT EVOQUE FREELANDER RANGE_ROVER RANGE_ROVER_SPORT RANGE_ROVER_VELAR RANGE_ROVER_VELAR LIFAN 10712
	BREEZ CEBRIUM CELLIYA MURMAN MYWAY SMILY SOLANO X50 X60 X70 X70 LIGIER 10884
	JS_51 JS_51 LINCOLN 10885
	AVIATOR LS MKC MKT NAUTILUS NAVIGATOR TOWN_CAR ZEPHYR ZEPHYR LTI 10925
	TX TX LUXGEN 10926
	7_SUV U7_TURBO U7_TURBO MAHINDRA 10934
	MARSHAL MARSHAL MASERATI 10935
	GHIBLI GRAN_TURISMO LEVANTE QUATTROP

In [30]:
# extra_train.to_csv('./Project_5_data/extra_train_2')

In [32]:
train = pd.read_csv('./Project_5_data/train')
extra_train_1 = pd.read_csv('./Project_5_data/extra_train_1')
extra_train_2 = pd.read_csv('./Project_5_data/extra_train_2')

In [34]:
all_marks = train.append([extra_train_1,extra_train_2], ignore_index=True)

In [37]:
all_marks.to_csv('./Project_5_data/all_marks')

In [36]:
all_marks.head()

,Unnamed: 0,bodytype,brand,car_url,color,engineDisplacement,enginePower,equipment_dict,fuel_type,mileage,...,numberOfDoors,productionDate,sell_id,vehicleTransmission,vendor,Владельцы,ПТС,Привод,Руль,offerprice
0,0,седан,SKODA,https://auto.ru/cars/used/sale/skoda/120/10858...,бежевый,1.2,53.0,{'Прочее': ['Полноразмерное запасное колесо']},Бензин,80000.0,...,NaN,1984.0,1085892952,механическая,EUROPEAN,2 владельца,Оригинал,задний,Левый,170000.0
1,1,универсал 3 дв.,SKODA,https://auto.ru/cars/used/sale/skoda/1200/1102...,голубой,1.2,45.0,NaN,Бензин,18000.0,...,3.0,1960.0,1102072444,механическая,EUROPEAN,1 владелец,Оригинал,задний,Левый,400000.0
2,2,хэтчбек 5 дв.,SKODA,https://auto.ru/cars/used/sale/skoda/fabia/110...,пурпурный,1.6,105.0,{'Безопасность': ['Подушка безопасности водите...,Бензин,168000.0,...,5.0,2009.0,1102546031,автоматическая,EUROPEAN,2 владельца,Оригинал,передний,Левый,439000.0
3,3,хэтчбек 5 дв.,SKODA,https://auto.ru/cars/used/sale/skoda/fabia/110...,белый,1.6,105.0,{'Безопасность': ['Антипробуксовочная система ...,Бензин,66165.0,...,5.0,2011.0,1102447624,автоматическая,EUROPEAN,1 владелец,Оригинал,передний,Левый,470000.0
4,4,хэтчбек 5 дв.,SKODA,https://auto.ru/cars/used/sale/skoda/fabia/110...,жёлтый,1.6,105.0,{'Безопасность': ['Антипробуксовочная система ...,Бензин,45748.0,...,5.0,2013.0,1101532994,автоматическая,EUROPEAN,3 или более,Оригинал,передний,Левый,487767.0


In [35]:
all_marks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64706 entries, 0 to 64705
Data columns (total 22 columns):
Unnamed: 0             64706 non-null int64
bodytype               64691 non-null object
brand                  64693 non-null object
car_url                64706 non-null object
color                  64693 non-null object
engineDisplacement     64568 non-null float64
enginePower            64568 non-null float64
equipment_dict         48186 non-null object
fuel_type              64568 non-null object
mileage                64693 non-null float64
modelDate              64691 non-null object
model_name             64693 non-null object
numberOfDoors          33163 non-null float64
productionDate         64693 non-null float64
sell_id                64706 non-null int64
vehicleTransmission    64691 non-null object
vendor                 64706 non-null object
Владельцы              64687 non-null object
ПТС                    64691 non-null object
Привод                 64691 non-

## Получение данных по одной модели одной марки

In [68]:
mark = list(marks_models_dict.keys())[11]
mark

models_list = marks_models_dict[mark]
model = models_list[0]

url = 'https://auto.ru/moskva/cars/' + mark + '/' + model + '/used/'
url

'https://auto.ru/moskva/cars/audi/100/used/'

In [69]:
columns = ['year','kmAge','bodytype','color','volume','power','fuel_type','transmission',
 'drive','wheel','state','ownersCount','pts','customs','complectation','offerprice']
trn = pd.DataFrame(columns=columns)

In [70]:
%%time

# просмотр последовательно всех страниц текущей модели текущей марки
# for pages_num in tqdm(range(1,10)): 
for pages_num in range(1,10): 

    
    if pages_num==1: params = {}
    else:            params = {'page': pages_num}
    
    res = requests.get(url, params=params, headers=headers)
    res.encoding = 'utf-8'
    time.sleep(1)
    
    # создаем обьект bs4.BeautifulSoup из html очередной страницы с обьявлениями по текущей модели текущей марки
    soup = BeautifulSoup(res.text, 'html.parser')  
    
    # список html-ек карточек на странице                                               
    tickets_on_page_list = soup.find_all('a', class_='ListingItemTitle-module__link') 
    
    if not tickets_on_page_list: # выход по исчерпанию страниц
        break
        
    # обработка карточек на странице
    for ticket in tickets_on_page_list:
        # получаем url карточки текущего обьявления
        ticket_url = ticket.get('href')
    
        # получем html карточки текщего обьявления
        ticket_res = requests.get(ticket_url)
        ticket_res.encoding = 'utf-8'
        time.sleep(0.1)
    
        # создаем обьект bs4.BeautifulSoup из html карточки текщего обьявления
        ticket_bs = BeautifulSoup(ticket_res.text, 'html.parser')
    
        # извлекаем признаки и заполняем строку train-а
        trn.loc[len(trn)] = get_features_from_ticket(ticket_bs)
    
    


 33%|██████████████▋                             | 3/9 [01:10<02:29, 24.98s/it]

Wall time: 1min 12s


In [71]:
trn

,year,kmAge,bodytype,color,volume,power,fuel_type,transmission,drive,wheel,state,ownersCount,pts,customs,complectation,offerprice
0,1993,786115,седан,синий,2.3,133,Бензин,механическая,передний,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,{'Элементы экстерьера': ['Легкосплавные диски'...,93000
1,1987,250000,седан,серый,1.8,88,"Бензин, газобаллонное оборудование",механическая,передний,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,{'Элементы экстерьера': ['Легкосплавные диски'...,75000
2,1990,241500,седан,красный,2.3,133,Бензин,механическая,передний,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,{'Салон': ['Регулировка передних сидений по вы...,100000
3,1988,250000,седан,серый,1.8,88,Бензин,механическая,передний,Левый,Не требует ремонта,3 или более,Дубликат,Растаможен,{},115000
4,1991,300000,седан,красный,2.0,115,Бензин,механическая,передний,Левый,Не требует ремонта,3 или более,Дубликат,Растаможен,{},130000
5,1992,350000,седан,белый,2.0,101,Бензин,механическая,передний,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,{},160000
6,1991,326000,седан,чёрный,2.3,133,Бензин,механическая,передний,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,{},180000
7,1992,320000,седан,серый,2.8,174,Бензин,механическая,полный,Левый,Не требует ремонта,3 или более,Дубликат,Растаможен,{},90000
8,1981,260000,седан,зелёный,1.6,85,Бензин,механическая,передний,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,{'Салон': ['Регулировка сиденья водителя по вы...,500000
9,1991,400000,седан,белый,2.3,133,Бензин,механическая,передний,Левый,Не требует ремонта,3 или более,Дубликат,Растаможен,{},155000


# Подвал

for loc in tqdm(range(0,len(unuseful_features_LR),1)):

In [21]:
hdrs_office = {
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
'Cache-Control': 'max-age=0',
'Connection': 'keep-alive',
'Cookie': 'suid=3e1833cc7469d2dc00ccae04449006c4.9491b97d8d423e5ce05ef1c79af8855d; _ym_uid=1599469037495440295; _ga=GA1.2.1600002130.1601280177; _csrf_token=1d7055285f2e0f66d88a7fe1fc75e2eec8ae6cd772464581; autoru_sid=a%3Ag60264c2d2igjpia4p3it7lul5qeu67s.cda7ff5877363e08fe2b0faf7c62fcdd%7C1613122605806.604800.gg8JxiHvE9AyBZihcInqEA.Ld4FRMZg9Xk8q0iFZFqjpOoMCJDw6N7itfobSBJS6WA; autoruuid=g60264c2d2igjpia4p3it7lul5qeu67s.cda7ff5877363e08fe2b0faf7c62fcdd; from=direct; X-Vertis-DC=vla; yuidlt=1; yandexuid=905825121577456505; my=YyYBAS4BAToBAQA%3D; counter_ga_all7=2; gdpr=0; _ym_isad=2; gids=213; _gid=GA1.2.204714820.1613122653; autoru-visits-count=1; from_lifetime=1613124329322; _ym_d=1613124329; cycada=QtZcIspPQofWD6yVq1tXyo1meqCq6CwJcSH8NY94K9c=',
'Host': 'auto.ru',
'Referer': 'https://auto.ru/moskva/cars/used/',
'sec-ch-ua': '"Chromium";v="88", "Google Chrome";v="88", ";Not A Brand";v="99"',
'sec-ch-ua-mobile': '?0',
'Sec-Fetch-Dest': 'document',
'Sec-Fetch-Mode': 'navigate',
'Sec-Fetch-Site': 'same-origin',
'Sec-Fetch-User': '?1',
'Upgrade-Insecure-Requests': '1',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; ,Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
},

hdrs_home = {
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
# 'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
# 'Connection': 'keep-alive',
'Cookie': 'suid=bf4a59ff8840503c2077bf716a7bbeef.295e234731ada0bc538a541689345348; tmr_lvid=d2115cf4bc0ee3e6bc3ad89db8fdac9d; tmr_lvidTS=1596630011564; _ym_uid=1596630015684249973; _ga=GA1.2.1113974574.1596630016; autoruuid=g5f949c622cu74bpb1qps6a12l6p5u7t.07262c19ce2a1f07cac53c4ce06ef69b; gids=213; gradius=200; mindboxDeviceUUID=a231f610-2f9e-4911-9cb8-9112afa13ab1; directCrm-session=%7B%22deviceGuid%22%3A%22a231f610-2f9e-4911-9cb8-9112afa13ab1%22%7D; tmr_reqNum=15; yuidlt=1; yandexuid=1471619051363959249; my=YysBgNU2AQEA; autoru_sid=a%3Ag5f949c622cu74bpb1qps6a12l6p5u7t.07262c19ce2a1f07cac53c4ce06ef69b%7C1612822568824.604800.vXs7yiSNjHUj9vs_NTRT7Q.Ab6eLoCklN5OVQvHKRzq30jYE9puDG83Hht-435SfG4; crookie=uCcId3jGlYfIvH+2UzzszOwTawIMGSzWumxanVb1Ras+Mu6qi+8yzj8EL+czftU9orxvkiKDKe/wgGsJDdJMlqyn6WU=; cmtchd=MTYxMjgyMjU3MTQ4MA==; _csrf_token=04c10009d8f17a08b4d507f70f2a287c28bf3c17f7b3ef42; X-Vertis-DC=vla; gdpr=0; _ym_isad=2; index-selector-tab=marks; listing_view_session={}; listing_view=%7B%22output_type%22%3Anull%2C%22version%22%3A1%7D; autoru-visits-count=2; salon_phone_utms=utm_medium%3Dcpm%26utm_source%3Dauto-ru%26utm_campaign%3Dauto-ru_rus-r225_proauto-rk2021%26utm_content%3D113pa-100PRx40-otchety-o-proshlom-mashiny-ot-99-rublei_proauto-promo-page_rus-r225; hide-proauto-pimple=1; from=direct; _ym_d=1613218878; from_lifetime=1613218878985; cycada=S1YNtBaS32C5Sgb39LwY/7150N9cGJGcL1yh7v8BaXc=',
'Host': 'auto.ru',
# 'sec-ch-ua': '"Chromium";v="88", "Google Chrome";v="88", ";Not A Brand";v="99"',
# 'sec-ch-ua-mobile': '?0',
# 'Sec-Fetch-Dest': 'document',
# 'Sec-Fetch-Mode': 'navigate',
# 'Sec-Fetch-Site': 'none',
# 'Sec-Fetch-User': '?1',
# 'Upgrade-Insecure-Requests': '1',
'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
}

In [ ]:
def features_parser (url_list):
    counter = 0
    for card_url in url_list:
        response = requests.get(card_url)
        response.encoding = 'utf-8'
        page = BeautifulSoup(response.text, 'html.parser')
    # body type
        body = page.find(itemprop = 'bodyType')
        if body is None:
            data_train.at[counter,'bodyType'] = None
        else:
            data_train.at[counter,'bodyType'] = body['content']
    #brand
        brand = page.find(itemprop = 'brand')
        if brand is None:
            data_train.at[counter, 'brand'] = None
        else:
            data_train.at[counter, 'brand'] = brand['content']
    # color
        color = page.find(itemprop = 'color')
        if color is None:
            data_train.at[counter,'color'] = None
        else:
            data_train.at[counter,'color'] = color['content']
    # fuel type
        fuel = page.find(itemprop = 'fuelType')
        if fuel is None:
            data_train.at[counter,'fuelType'] = None
        else:
            data_train.at[counter,'fuelType'] = fuel['content']
    # model date
        model_date = page.find(itemprop = 'modelDate')
        if model_date is None:
            data_train.at[counter,'modelDate'] = None
        else:
            data_train.at[counter,'modelDate'] = model_date['content']
    # number of doors
        doors = page.find(itemprop = 'numberOfDoors')
        if doors is None:
            data_train.at[counter,'numberOfDoors'] = None
        else:
            data_train.at[counter,'numberOfDoors'] = doors['content']
    # production date
        prod_date = page.find(itemprop = 'productionDate')
        if prod_date is None:
            data_train.at[counter,'productionDate'] = None
        else:
            data_train.at[counter,'productionDate'] = prod_date['content']
    # vehicle transmission
        trans = page.find(itemprop = 'vehicleTransmission')
        if trans is None:
            data_train.at[counter,'vehicleTransmission'] = None
        else:
            data_train.at[counter,'vehicleTransmission'] = trans['content']
    # engine displacement
        engine_ltr = page.find(itemprop = 'engineDisplacement')
        if engine_ltr is None:
            data_train.at[counter,'engineDisplacement'] = None
        else:
            data_train.at[counter,'engineDisplacement'] = engine_ltr['content']
    # engine power
        engine_pwr = page.find(itemprop = 'enginePower')
        if engine_pwr is None:
            data_train.at[counter,'enginePower'] = None
        else:
            data_train.at[counter,'enginePower'] = engine_pwr['content']
    # mileage
        mileage = page.find(class_ = 'CardInfoRow CardInfoRow_kmAge')
        if mileage is None:
            data_train.at[counter,'mileage'] = None
        else:
            mileage = mileage.text.replace('Пробег', '')
            mileage = mileage.replace('\xa0', '')
            mileage = mileage.replace('км', '')
            data_train.at[counter,'mileage'] = mileage
    # drive type
        drive = page.find(class_ = 'CardInfoRow CardInfoRow_drive')
        if drive is None:
            data_train.at[counter,'Привод'] = None
        else:
            drive = drive.text.replace('Привод', '')
            data_train.at[counter,'Привод'] = drive
    # wheel
        wheel = page.find(class_ = 'CardInfoRow CardInfoRow_wheel')
        if wheel is None:
            data_train.at[counter,'Руль'] = None
        else:
            wheel = wheel.text.replace('Руль', '')
            data_train.at[counter,'Руль'] = wheel
    # owners
        owners = page.find(class_ = 'CardInfoRow CardInfoRow_ownersCount')
        if owners is None:
            data_train.at[counter,'Владельцы'] = None
        else:
            owners = owners.text.replace('Владельцы', '')
            owners = owners.replace('\xa0', '')
            data_train.at[counter,'Владельцы'] = owners
    # vehicle certificate
        certificate = page.find(class_ = 'CardInfoRow CardInfoRow_pts')
        if certificate is None:
            data_train.at[counter,'ПТС'] = None
        else:
            certificate = certificate.text.replace('ПТС', '')
            data_train.at[counter,'ПТС'] = certificate
    # price
        price = page.find(class_ = 'OfferPriceCaption__price')
        if price is None:
            data_train.at[counter,'price'] = None
        else:
            price = price.text.replace('\xa0', '')
            price = price.replace('₽', '')
            data_train.at[counter,'price'] = price
    # counter up
        counter += 1

In [72]:
{
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
'Connection': 'keep-alive',
'Cookie': 'suid=bf4a59ff8840503c2077bf716a7bbeef.295e234731ada0bc538a541689345348; tmr_lvid=d2115cf4bc0ee3e6bc3ad89db8fdac9d; tmr_lvidTS=1596630011564; _ym_uid=1596630015684249973; _ga=GA1.2.1113974574.1596630016; autoruuid=g5f949c622cu74bpb1qps6a12l6p5u7t.07262c19ce2a1f07cac53c4ce06ef69b; gids=213; gradius=200; mindboxDeviceUUID=a231f610-2f9e-4911-9cb8-9112afa13ab1; directCrm-session=%7B%22deviceGuid%22%3A%22a231f610-2f9e-4911-9cb8-9112afa13ab1%22%7D; tmr_reqNum=15; yuidlt=1; yandexuid=1471619051363959249; my=YysBgNU2AQEA; autoru_sid=a%3Ag5f949c622cu74bpb1qps6a12l6p5u7t.07262c19ce2a1f07cac53c4ce06ef69b%7C1612822568824.604800.vXs7yiSNjHUj9vs_NTRT7Q.Ab6eLoCklN5OVQvHKRzq30jYE9puDG83Hht-435SfG4; crookie=uCcId3jGlYfIvH+2UzzszOwTawIMGSzWumxanVb1Ras+Mu6qi+8yzj8EL+czftU9orxvkiKDKe/wgGsJDdJMlqyn6WU=; cmtchd=MTYxMjgyMjU3MTQ4MA==; _csrf_token=04c10009d8f17a08b4d507f70f2a287c28bf3c17f7b3ef42; X-Vertis-DC=vla; gdpr=0; _ym_isad=2; index-selector-tab=marks; listing_view_session={}; listing_view=%7B%22output_type%22%3Anull%2C%22version%22%3A1%7D; autoru-visits-count=2; salon_phone_utms=utm_medium%3Dcpm%26utm_source%3Dauto-ru%26utm_campaign%3Dauto-ru_rus-r225_proauto-rk2021%26utm_content%3D113pa-100PRx40-otchety-o-proshlom-mashiny-ot-99-rublei_proauto-promo-page_rus-r225; hide-proauto-pimple=1; from=direct; _ym_d=1613218878; from_lifetime=1613218878985; cycada=S1YNtBaS32C5Sgb39LwY/7150N9cGJGcL1yh7v8BaXc=',
'Host': 'auto.ru',
'sec-ch-ua': '"Chromium";v="88", "Google Chrome";v="88", ";Not A Brand";v="99"',
'sec-ch-ua-mobile': '?0',
'Sec-Fetch-Dest': 'document',
'Sec-Fetch-Mode': 'navigate',
'Sec-Fetch-Site': 'none',
'Sec-Fetch-User': '?1',
'Upgrade-Insecure-Requests': '1',
'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
};

In [3]:
'https://auto.ru/catalog/cars/used/?view_type=list'
'https://auto.ru/catalog/cars/used/?page_num=2&view_type=list'

'https://auto.ru/catalog/cars/used/?page_num=2&view_type=list'

In [63]:
%%time

# просмотр последовательно всех страниц текущей модели текущей марки
for pages_num in tqdm(range(1,10)): 
    
    if pages_num==1: params = {}
    else:            params = {'page': pages_num}
    
    res = requests.get(url, params=params, headers=headers)
    res.encoding = 'utf-8'
    time.sleep(1)
    
    # создаем обьект bs4.BeautifulSoup из html очередной страницы с обьявлениями по текущей модели текущей марки
    soup = BeautifulSoup(res.text, 'html.parser')  
    
    # список html-ек карточек на странице                                               
    tickets_on_page_list = soup.find_all('a', class_='ListingItemTitle-module__link') 
    
    if not tickets_on_page_list: # выход по исчерпанию страниц
        break
        
    # обработка карточек на странице
    for ticket in tickets_on_page_list:
        # получаем url карточки текущего обьявления
        ticket_url = ticket.get('href')
    
        # получем html карточки текщего обьявления
        ticket_res = requests.get(ticket_url)
        ticket_res.encoding = 'utf-8'
        time.sleep(0.1)
    
        # создаем обьект bs4.BeautifulSoup из html карточки текщего обьявления
        ticket_bs = BeautifulSoup(ticket_res.text, 'html.parser')
    
        # извлекаем признаки и заполняем строку train-а
        trn.loc[len(trn)] = get_features_from_ticket(ticket_bs)

### ==========================================================================

In [ ]:
headers = ''' Host: auto.ru User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:70.0) 
Gecko/20100101 Firefox/70.0 Accept: / Accept-Language: ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3 
Accept-Encoding: gzip, deflate, br Referer: https://auto.ru/moskva/cars/all/?year_from=2000&output_type=list&page=1 
x-client-app-version: 202012.01.170752 x-page-request-id: 63c3e93995f1ef12b4ddf35e9e9ff750 
x-client-date: 1606907658457 x-csrf-token: 5c191babb10a904dee2fd6280a6b4b4a2d5e8f32849d2292 
x-requested-with: fetch content-type: application/json Origin: https://auto.ru 
Content-Length: 84 Connection: keep-alive 
Cookie: suid=d813bc26bba4325c1d7533623696d743.10bff5a6232dc784e2af84bda5b80129; 
autoru_sid=a%3Ag5fbb6843277sltq6dq3o0hdn0m4va4a.29d32c68d00774244ad8800a0c319af4%7C1606889088872.
604800.pTPOzopdO-Fd5_4rHKw-VA.9WiQQIf2GKFuBmA4W8M3KMonJTGKl0C8E4WiikWI9UM; 
autoruuid=g5fbb6843277sltq6dq3o0hdn0m4va4a.29d32c68d00774244ad8800a0c319af4; 
yuidcs=1; yandexuid=56446481587545136; my=YwA%3D; 
_csrf_token=5c191babb10a904dee2fd6280a6b4b4a2d5e8f32849d2292; 
from_lifetime=1606907657108; from=direct; 
salon_phone_utms=utm_medium%3Dcpc%26utm_source%3Dyandex_direct%26utm_campaign%3D48297426_ysearch_msk-r1_brand-netochnoe%26utm_content%3Davtoru-brand_morda_view_exp-no_msk-r1; 
yuidlt=1; crookie=srlYBXe4eEBR2wBfpK6UJiL9XNMKkFA4eUD7SZeSU3ce+oHX7LUHTdckoGSTNy+knzz6CDuwXy9b+HQfOgI9KkfQfKY=; 
cmtchd=MTYwNjg4OTA5MjAzNQ==; gdpr=0; cycada=KxTHT7+qDaLY6c2tr2nHhQV2hnjxKHol2l9mSC4j9H0=; 
_ym_uid=1606889095445538116; _ym_d=1606907657; _ym_isad=2; X-Vertis-DC=vla; 
_ym_visorc_22753222=b; _ym_visorc_148422=w; _ym_visorc_588551=w '''.strip().split("\n")

url = "https://auto.ru/-/ajax/desktop/listingSpecial/" 

dict_headers = {} for header in headers: key, value = header.split(': ') dict_headers[key] = value

In [ ]:
offers = [] 
for x in range(2000, 2020): 
    for y in range (1,100): 
        param = { "section":"all", "category":"cars", "geo_radius":200, "geo_id":[213], "page": y, "year_from": x} 
        response = requests.post(url, json=param, headers = dict_headers) 
        data = response.json() offers.extend(data['offers']) 
        print("current year: ", x, "current page: ", y)

### ==========================================================================

In [ ]:
# Парсинг проводился по слующим критериям 'bodyType', 'brand', 'color', 
#'fuelType', 'modelDate', 'model_name', 'numberOfDoors', 'productionDate', 'vehicleConfiguration', 'vehicleTransmission', 'price', 'name', 'engineDisplacemen', 'enginePower', 'mileage'

"""

model_all = ['BMW', 'VOLKSWAGEN', 'NISSAN', 'MERCEDES', 'TOYOTA', 'AUDI', 'MITSUBISHI', 'SKODA', 
             'VOLVO', 'HONDA', 'INFINITI', 'LEXUS']

auto_list_long =[]

for brand in model_all:  # марка авто
    print(brand)
    for i in range(1, 1200):     # кол-во страниц для парсинга с авто.ру   
        print(i)
        response = rs.get(f'https://auto.ru/moskva/cars/{brand}/used/?output_type=list&page={i}')
        if response.status_code != 200:
            raise BaseException("response code" + str(response.status_code))
        response.encoding = 'utf-8'
        soup = BS(response.text, 'html.parser')
        page = soup.find_all('div', class_='ListingItem-module__container')  
        for k in range(len(page)):     # цикл для прохода по объявлениям в одной странице
            auto_html = page[k].find_all('meta')
            auto_list = []
            for i in range(len(auto_html)):   # цикл для обработки отдельного объявления на странице
                auto_list.append((str(auto_html[i])[15:]).split('"', 1)[0])
                
            # Убираем лишние параметры автомобиля из списка    
            pos_out = {4,11,12,14,15,19}
            auto_list_short = []
            for n in range(len(auto_list)):   # цикл для удаления лишних параметров из объявления
                if n not in pos_out:
                    auto_list_short.append(auto_list[n].replace('\xa0', ' ')) 
        
            # Считываем пробег(он расположен отдельно от других данных)    
            all_km = page[k].find('div', class_='ListingItem-module__kmAge')
            xx = str(all_km)[39:].split('<', 1)[0].replace('\xa0', ' ')
            auto_list_short.append(str(xx))    # добавляем пробег
            auto_list_long.append(auto_list_short)    # добавляем список пар-ров автомобиля в список списков

### ==========================================================================

In [ ]:
auto_list_long =[] # здесь соберем список списков(эл-т - список параметров конкретного автомобиля)
for brand in ['MERCEDES']:  # марка авто
    
    for i in range(1, 1200):     # кол-во страниц для парсинга с авто.ру   
        response = requests.get(f'https://auto.ru/moskva/cars/{brand}/used/?output_type=list&page={i}')
        if response.status_code != 200:
            raise BaseException("response code" + str(response.status_code))
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')
        page = soup.find_all('div', class_='ListingItem-module__container')  
        for k in range(len(page)):     # цикл для прохода по объявлениям в одной странице
            auto_html = page[k].find_all('meta')
            auto_list = []
            for i in range(len(auto_html)):   # цикл для обработки отдельного объявления на странице
                auto_list.append((str(auto_html[i])[15:]).split('"', 1)[0])
                
            # Убираем лишние параметры автомобиля из списка    
            pos_out = {4,11,12,14,15,19}
            auto_list_short = []
            for n in range(len(auto_list)):   # цикл для удаления лишних параметров из объявления
                if n not in pos_out:
                    auto_list_short.append(auto_list[n].replace('\xa0', ' ')) 
        
            # Считываем пробег(он расположен отдельно от других данных)    
            all_km = page[k].find('div', class_='ListingItem-module__kmAge')
            xx = str(all_km)[39:].split('<', 1)[0].replace('\xa0', ' ')
            auto_list_short.append(str(xx))    # добавляем пробег
            auto_list_long.append(auto_list_short)    # добавляем список пар-ров автомобиля в список списков


### ==========================================================================


In [ ]:
def get_features_from_ticket(ticket_bs):
    '''
    возвращает список с признакамии из карточки обьявления
    
    ticket_bs: bs4.BeautifulSoup
        обьект BeautifulSoup полученный из html карточки обьявления
        
    '''
    # получение значений признаков (кроме комплектации) из карточки текущего обьявления
    brand = ticket_bs.find_all('a',class_='CardBreadcrumbs__itemText')[0].text.strip()
    car_url = 0
    equipment_dict = 0
    modelDate = ticket_bs.find_all('a',class_='CardBreadcrumbs__itemText')[2].text.strip() ## !!!!!!!!! ОБРАБОТАТЬ !!!!!
    model_name = 0
    numberOfDoors = 0
    productionDate = 0
    vendor = 0
    
    
    
    
    year = ticket_bs.find('li',class_='CardInfoRow_year').find('a').text
    mileage = ticket_bs.find('li',class_='CardInfoRow_kmAge').find_all('span')[1].text
    mileage = re.sub("\D", "", kmAge)
    bodytype = ticket_bs.find('li',class_='CardInfoRow_bodytype').find('a').text
    color = ticket_bs.find('li',class_='CardInfoRow_color').find('a').text
    engineDisplacement = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[0]
    engineDisplacement =  re.sub("[^\d.]", "", volume)
    enginePower = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[1]
    enginePower = re.sub("\D", "", power)
    fuel_type = ticket_bs.find('li',class_='CardInfoRow_engine').find('div').text.split(' / ')[2]
    vehicleTransmission = ticket_bs.find('li',class_='CardInfoRow_transmission').find_all('span')[1].text
    drive = ticket_bs.find('li',class_='CardInfoRow_drive').find_all('span')[1].text
    wheel = ticket_bs.find('li',class_='CardInfoRow_wheel').find_all('span')[1].text
    state = ticket_bs.find('li',class_='CardInfoRow_state').find_all('span')[1].text
    ownersCount = ticket_bs.find('li',class_='CardInfoRow_ownersCount').find_all('span')[1].text
    pts = ticket_bs.find('li',class_='CardInfoRow_pts').find_all('span')[1].text
    customs = ticket_bs.find('li',class_='CardInfoRow_customs').find_all('span')[1].text
    
    # создание словаря комплектации
    complectation_dict = {}
    complectation = ticket_bs.find('section',class_='CardComplectation')
    if complectation:
        complectation_items = complectation.find_all('div',class_="ComplectationGroups__group")
        for item in complectation_items:
            item_name = item.find('span',class_="ComplectationGroups__itemName").text
            item_content = [x.text for x in item.find_all('li',class_="ComplectationGroups__itemContentEl")]
            complectation_dict[item_name] = item_content
        
    # получение цены предложения
    offerprice = ticket_bs.find('span',class_='OfferPriceCaption__price').text
    offerprice = re.sub("\D", "", offerprice)
    
    # заполняем строку значенями
#     ind = ['year','kmAge' ,'bodytype' ,'color' ,'volume' ,'power' ,'fuel_type','transmission',
#      'drive','wheel','state','ownersCount','pts','customs','complectation','offerprice']
    vol = [year,kmAge ,bodytype ,color ,engineDisplacement ,enginePower ,fuel_type,vehicleTransmission,
           drive,wheel,state,ownersCount,pts,customs,complectation_dict,offerprice]
#     data_row = pd.Series(data=vol,index=ind)
    
    return vol

In [16]:
model_url = 'https://auto.ru/moskva/cars/audi/100/used/'

get_model_generation_year(model_url)

In [ ]:
ticket_res = requests.get(ticket_url)
ticket_res.encoding = 'utf-8'

ticket_bs = BeautifulSoup(ticket_res.text, 'html.parser')

(ticket_bs.find_all('a',class_='CardBreadcrumbs__itemText')[0].text.strip() + ' ' +
 ticket_bs.find_all('a',class_='CardBreadcrumbs__itemText')[1].text.strip() + ' ' +
 ticket_bs.find_all('a',class_='CardBreadcrumbs__itemText')[2].text.strip()
)

In [ ]:
%%time
# создаем pd.Series с индексом - обозначениеми марки и занчениями - списками моделей этой марки  
marks_models_dict = pd.Series() 

# for pages_num in tqdm(range(1,20)):
for pages_num in range(1,20):
    
    if pages_num==1: params = {'view_type': 'list'}
    else:            params = {'page_num': pages_num,'view_type': 'list'}
    
    res = requests.get(url_for_mmdict, params=params, headers=headers)
    time.sleep(1)
    
    soup = BeautifulSoup(res.text, 'html.parser')  # создаем обьект bs4.BeautifulSoup
    marks_on_page_list = soup.find_all('dd', class_='catalog-all-text-list__desc') # список html-ек марок на странице
    
    if not marks_on_page_list: 
#         print ('стоп')
        break
        
    for mark in marks_on_page_list:
        models_of_mark = mark.find_all('a', class_='link_theme_auto') # список html-ек моделей марки
        
        link_for_mark_name = models_of_mark[0].get('href')  # линк первой модели, содержит обозначене марки на auto.ru
        mark_start = link_for_mark_name.find('cars/') + 5   # первый символ обозначеня марки
        mark_end = link_for_mark_name.find('/', mark_start) # последний символ обозначения марки
        mark_name = link_for_mark_name[mark_start:mark_end] # получение обозначения марки      
        
        models_list = []
        for model in models_of_mark:
            link_theme_auto = model.get('href')                 # линк модели, содержит обозначение модели на auto.ru
            model_start = mark_end+1                            # первый символ обозначения модели
            model_end = link_theme_auto.find('/', model_start)  # последний символ обозначения модели
            model_name = link_theme_auto[model_start:model_end] # получение обозначения модели
            models_list.append(model_name)
      
        marks_models_dict[mark_name] = models_list